# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

import json



from api_keys import geoapify_key




In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,-34.23,94,31,2.87,RU,1705128383
1,1,stanthorpe,-28.6667,151.9500,24.68,69,100,4.99,AU,1705128383
2,2,port-aux-francais,-49.3500,70.2167,7.02,91,73,10.79,TF,1705128383
3,3,oakdale,44.9630,-92.9649,-10.18,85,100,8.75,US,1705128174
4,4,linqiong,30.4159,103.4609,13.90,52,88,1.60,CN,1705128385


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:

#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat",geo = True, tiles = "OSM", size = "Humidity", frame_width = 800, frame_height = 600, scale = 0.5, color = "City")


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
#Still using metric (C)

# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

# Get the max teperature.
max_temp =  city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [62]:
min_temp = city_data_df["Max Temp"] > 20
max_temp = city_data_df["Max Temp"] < 30
wind_speed = city_data_df["Wind Speed"] > 5

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:


city_temp_range_df = city_data_df[min_temp & max_temp & wind_speed]

# Drop any rows with null values & Display sample data
city_temp_range_df.dropna()
city_temp_range_df





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,utrik,11.2278,169.8474,26.80,77,6,9.41,MH,1705128386
20,20,adamstown,-25.0660,-130.1015,23.77,81,87,7.46,PN,1705128392
33,33,kingston,17.9970,-76.7936,26.53,80,20,5.66,JM,1705128321
46,46,papatowai,-46.5619,169.4708,20.04,64,100,5.44,NZ,1705128403
53,53,kulhudhuffushi,6.6221,73.0700,27.80,76,100,7.62,MV,1705128405
...,...,...,...,...,...,...,...,...,...,...
510,510,viedma,-40.8135,-62.9967,20.55,72,97,8.63,AR,1705128581
528,528,papao,-17.7500,-149.5500,26.82,82,51,6.66,PF,1705128586
540,540,sarakhs,36.5449,61.1577,20.42,18,95,8.02,IR,1705128589
562,562,bac lieu,9.2850,105.7244,29.91,55,100,7.30,VN,1705128596


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity


#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API



hotel_df = city_temp_range_df.copy()

columns_to_drop = ['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date']

hotel_df = hotel_df.drop(columns=columns_to_drop)

hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
8,utrik,11.2278,169.8474,77,MH,
20,adamstown,-25.0660,-130.1015,81,PN,
33,kingston,17.9970,-76.7936,80,JM,
46,papatowai,-46.5619,169.4708,64,NZ,
53,kulhudhuffushi,6.6221,73.0700,76,MV,
...,...,...,...,...,...,...
510,viedma,-40.8135,-62.9967,72,AR,
528,papao,-17.7500,-149.5500,82,PF,
540,sarakhs,36.5449,61.1577,18,IR,
562,bac lieu,9.2850,105.7244,55,VN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit":20}
   

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"    
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"     
    
    name_address = requests.get(base_url, params=params)
    
    #Convert to JSON
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
        

Starting hotel search
utrik - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
papatowai - nearest hotel: No hotel found
kulhudhuffushi - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
carnarvon - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
bandarbeyla - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
callao - nearest hotel: Casa Ronald
port elizabeth - nearest hotel: Waterford Hotel
nuku'alofa - nearest hotel: City Hotel
namdrik - nearest hotel: No hotel found
cadale - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
port mathurin - nearest hotel: Escale Vacances
afaahiti - nearest hotel: Omati Lodge
marsh harbour - nearest hotel: Lofty Fig Villas
fort bragg - nearest hotel: Airborne Inn Lodging
gereida - nearest hotel: No hotel found
nyala - nearest hotel: سوني سفاري
itoman - nearest hotel: No hotel found
kismayo - nearest hotel: 

,City,Lat,Lng,Humidity,Country,Hotel Name
8,utrik,11.2278,169.8474,77,MH,No hotel found
20,adamstown,-25.0660,-130.1015,81,PN,No hotel found
33,kingston,17.9970,-76.7936,80,JM,Aphrodite Hotel
46,papatowai,-46.5619,169.4708,64,NZ,No hotel found
53,kulhudhuffushi,6.6221,73.0700,76,MV,No hotel found
...,...,...,...,...,...,...
510,viedma,-40.8135,-62.9967,72,AR,Níjar
528,papao,-17.7500,-149.5500,82,PF,Hiti Moana Villa Lodge
540,sarakhs,36.5449,61.1577,18,IR,هتل ابریشم
562,bac lieu,9.2850,105.7244,55,VN,Khách sạn Bạc Liêu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["Hotel Name","Country"])
# Display the map plot
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)